# EP1 - Mandelbrot set
###### Notebook baseado no dos miniEPs

Preencha o nome dos 5 membros do seu grupo na tabela abaixo:

| Nome | NUSP |
|------|------|
| Caio Andrade | 9797232 |
| Caio Fontes | 10692061 |
| Eduardo Laurentino | 8988212 |
| Thiago Teixeira | 10736987 |
| Washington Meireles | 10737157 |

In [ ]:
] up

In [ ]:
] st

## Tarefa 3 - Apresentação dos resultados

### Funções para compilar/Rodar os testes:

In [ ]:
;make 

In [ ]:
;./mandelbrot_seq

In [ ]:
; ./mandelbrot_seq 0.175 0.375 -0.1 0.1 200 0

In [ ]:
;./mandelbrot_pth

In [ ]:
;./mandelbrot_omp

In [ ]:
using DataFrames, Query, StatsPlots, Statistics

function measure_mandelbrot(size, f, typ; thread = 0)
    if f==0 par = `-2.5 1.5 -2.0 2.0` #Full Picture
    elseif f==1 par = `-0.8 -0.7 0.05 0.15` #Seahorse Valley
    elseif f==2 par = `0.175 0.375 -0.1 0.1` #Elephant Valley
    elseif f==3 par = `-0.188 -0.012 0.554 0.754` #Triple Spiral Valley
    end                
    if thread == 0 
        results = parse.(Float64,
            chomp(read(`./$typ $par $size`, String)))
    else
        results = parse.(Float64,
            chomp(read(`./$typ $par $size $thread`, String)))
    end
        
    return DataFrame(size = size,
        f = f,
        threads = thread,
        duration = results[1])
end

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`, `method` e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [ ]:
function run_experiments(size, f, method, repetitions; threads = [])
    run(`make $method`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        duration = Float64[])  
    
    if threads == []
        for r in 1:repetitions
            for s in size
                append!(results,
                    measure_mandelbrot(s, f, method))    
            end
        end
    else
        for t in threads
            for s in size
                for r in 1:repetitions
                    append!(results,
                        measure_mandelbrot(s, f, method, thread = t))
                    end
                end
            end
        end
    return results
end

A função `parse_results` recebe um `DataFrame` de resultados, produzido pela função `run_experiments`. A função devolve um `DataFrame` com a média e o intervalo de confiança da média a 95% das estimativas e dos tempos de execução, agrupados por número de threads.

In [ ]:
function parse_results(results)
    parsed_results = results |>
                    @groupby({_.threads,_.size}) |>
                    @map({threads = key(_).threads,
                          size = _.size[1],
                          mean_duration = mean(_.duration),
                          ci_duration = 1.96 * std(_.duration)}) |>
                    DataFrame
    
    return parsed_results
end

### Geração de gráficos:

In [ ]:

function plot_results(x, y1, series_label1, yerror1;
                        y2 = [], series_label2 = [], yerror2 = [],
                        y3 = [], series_label3 = [], yerror3 = [],
                        y4 = [], series_label4 = [], yerror4 = [],
                        y5 = [], series_label5 = [], yerror5 = [])   
    p = scatter(x,        
            y1,
            yerror = yerror1,
            alpha = 0.6,
            labels = series_label1,
            xlabel = "threads",
            legend = :topleft)
    
    if y2 != []
        p = scatter(x,        
            y2,
            yerror = yerror2,
            alpha = 0.6,
            labels = series_label2,
            legend = :topleft)
    end
    if y3 != []
        p = scatter(x,        
            y3,
            yerror = yerror3,
            alpha = 0.6,
            labels = series_label3,
            legend = :topleft)
    end
    if y4 != []
        p = scatter(x,        
            y4,
            yerror = yerror4,
            alpha = 0.6,
            labels = series_label4,
            legend = :topleft)
    end
    if y5 != []
        p = scatter(x,        
            y5,
            yerror = yerror5,
            alpha = 0.6,
            labels = series_label5,
            legend = :topleft)
    end
    return p
end


## Realizando os experimentos:

### DataFrames:

Parametros a serem utilizados(usamos os mesmos do script `run_measurements.sh`):

In [ ]:
size = [2 ^ i for i in 4:13] #resolucao
thread = [2 ^ i for i in 0:5]
repetitions = 10;

#### Sequencial:

In [ ]:
fileName = "mandelbrot_seq"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions)
seq_full = parse_results(results)

In [ ]:
#Seahorse valley
results = run_experiments(size, 1, fileName , repetitions)
seq_seahorse = parse_results(results)

In [ ]:
#Elephant valley
results = run_experiments(size, 2, fileName, repetitions)
seq_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions)
seq_tripleSpiral = parse_results(results)

#### PThreads:

In [ ]:
fileName = "mandelbrot_pth"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions, threads=thread)
pth_full = parse_results(results)

In [ ]:
#Seahorse valley
results = run_experiments(size, 1, fileName, repetitions, threads=thread)
pth_seahorse = parse_results(results)

In [ ]:
#Elephant valley
results = run_experiments(size, 2, fileName, repetitions, threads=thread)
pth_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions, threads=thread)
pth_tripleSpiral = parse_results(results)

#### OpenMP:

In [ ]:
fileName = "mandelbrot_omp"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions, threads=thread)
omp_full = parse_results(results)

In [ ]:
#Seahores Valley
results = run_experiments(size, 1, fileName, repetitions, threads=thread)
omp_seahorse = parse_results(results)

In [ ]:
#Elephant Valley
results = run_experiments(size, 2, fileName, repetitions, threads=thread)
omp_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions, threads=thread)
omp_tripleSpiral = parse_results(results)

#### Salvando em .csv:

In [ ]:
using CSV
function save_csv_results(parsed_results, name)
    CSV.write(string("/dataBases", name, ".csv"), parsed_results)
end

In [ ]:
#Sequencial
save_csv_results(seq_full, "seq_full")
save_csv_results(seq_seahorse, "seq_seahorse")
save_csv_results(seq_elephant, "seq_elephant")
save_csv_results(seq_tripleSpiral, "seq_tripleSpiral")

In [ ]:
#Pthreads
save_csv_results(pth_full, "pth_full")
save_csv_results(pth_seahorse, "pth_seahorse")
save_csv_results(pth_elephant, "pth_elephant")
save_csv_results(pth_tripleSpiral, "pth_tripleSpiral")

In [ ]:
#OpenMP
save_csv_results(omp_full, "omp_full")
save_csv_results(omp_seahorse, "omp_seahorse")
save_csv_results(omp_elephant, "omp_elephant")
save_csv_results(omp_tripleSpiral, "omp_tripleSpiral")

## Gráficos: